In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from matplotlib.colors import to_rgb
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import train_test_split



import torch
import matplotlib.pyplot as plt

# Parameters
n_samples = 200

# # Sample x from uniform distribution in [-1, 1]
# x = 2 * torch.rand(n_samples, 1) - 1  # shape (n_samples, 1), range [-1, 1]

# # Labels: y = x^2
# y = x ** 2  # shape (n_samples, 1)

# # Plot for verification
# plt.scatter(x.numpy(), y.numpy(), alpha=0.6)
# plt.xlabel("x")
# plt.ylabel("y = x^2")
# plt.title("Sampled Data from [-1, 1] with Labels x^2")
# plt.grid(True)
# plt.show()

def make_x2in1d(output_dim, n_samples = 100, plot = True, batch_size = 128, filename = None):
    """Generates xor
    """
    # Generate training data
    # set random seed for reproducibility
    seed = np.random.randint(1000)
    print(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    
    data = 2 * torch.rand(n_samples, 1) - 1  # shape (n_samples, 1), range [-1, 1]
    labels = data ** 2  # shape (n_samples, 1)
    # Generate outer ring points
    
    
    if plot:
                # Plot for verification
        plt.scatter(data.numpy(), labels.numpy(), alpha=0.6)
        plt.xlabel("x")
        plt.ylabel("y = x^2")
        plt.title("Sampled Data from [-1, 1] with Labels x^2")
        plt.grid(True)
        plt.show()
        
                # Save plot if filename provided
        if filename is not None:
            plt.savefig(f'{filename}.png', bbox_inches='tight', dpi=300)
            print(f'Plot saved as {filename}.png')
        
        plt.show()
    
    # Convert to tensors
    # data_tensor = torch.tensor(data, dtype=torch.float32)

    labels_tensor = torch.tensor(labels, dtype=torch.float32)


    data = torch.tensor(data, dtype=torch.float32)
    labels = torch.tensor(labels, dtype=torch.float32) 
    labels = torch.tensor(labels.reshape(-1, 1), dtype=torch.float32)
   
    # Create DataLoader for training data
    train_dataset = TensorDataset(data, labels)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
    return train_dataloader

train_loader = make_x2in1d(1, n_samples=n_samples)

In [ ]:
for X, y in train_loader:
    print("Batch X:", X[:5])
    print("Batch y:", y[:5])
    break  # Just to show the first batch

In [ ]:
# to reload models.resnet module after changes without restarting the kernel
import importlib
import models.resnets
import models.training
importlib.reload(models.resnets) # Reload the module
importlib.reload(models.training) # Reload the module
from models.resnets import ResNet
from models.training import compute_accuracy, train_model, train_until_threshold, plot_loss_curve

In [ ]:
# Model Params
num_hidden = 10 # number of hidden layers. The total network has additionl 2 layers: input to hidden and hidden to output
input_dim = 1
hidden_dim = 1
output_dim = 1
activation = 'tanh' #'relu' and 'tanh' are supported

# Training Params
load_file = None
cross_entropy = False #True supported with binary classification only
num_epochs = 300

In [ ]:
model_base, acc_base, losses_base = train_until_threshold(ResNet,
    train_loader, train_loader,
    load_file = load_file, max_retries=1, threshold=0.95, cross_entropy=cross_entropy,
    input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_hidden=num_hidden, skip_param=0, activation=activation, early_stopping=False
)

plot_loss_curve(losses_base, title=f"Base Model Loss Curve", filename = 'ff1d')

input = torch.linspace(-1, 1, 100)
output = model_base(input.unsqueeze(1)).detach().numpy() 

plt.plot(input, output)

In [ ]:
test = torch.tensor([[0.0], [0.1]]) # test point
model_base(test).detach().numpy() # should be close to 0

loss = torch.nn.MSELoss()
test_loss = loss(model_base(test), torch.tensor([[0.0], [0.1]]))
print(f'Test loss: {test_loss.item()}')

In [ ]:
for X, y in train_loader:
    print("Batch X:", X[:5])
    print("Batch y:", y[:5])
    test = torch.tensor([[0.0], [0.1]]) # test point
    print("Model output for test point:", model_base(test).detach().numpy())
    loss_value = loss(model_base(test), torch.tensor([[0.0], [0.1]]))
    print("Loss for test point:", loss_value.item())
    break  # Just to show the first batch